In [1]:
import os
import sys
sys.path.append('../../')

import arboreto.core as ac

from arboreto.core import *
from arboreto.utils import *
from arboreto.algo import *

from memory_profiler import profile

import matplotlib.pyplot as plt
import distributed

In [2]:
distributed.__version__

'1.21.8'

In [3]:
macosko_40k = '/media/tmo/data/work/datasets/macosko/in/macosko_40k.tsv.gz'
tfs = '/media/tmo/data/work/datasets/macosko/in/mm9_TFs.upper.txt'
out = 'mac_40k.tsv'

In [4]:
from dask.distributed import Client, LocalCluster

In [5]:
client = Client(LocalCluster(memory_limit='10GB'))

In [6]:
client

Client Scheduler: tcp://127.0.0.1:35151 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 88 Cores: 88 Memory: 880.00 GB


In [7]:
create_graph??

Signature: create_graph(expression_matrix, gene_names, tf_names, regressor_type, regressor_kwargs, client, target_genes='all', limit=None, include_meta=False, early_stop_window_length=25, repartition_multiplier=1, seed=666)
Source:   
def create_graph(expression_matrix,
                 gene_names,
                 tf_names,
                 regressor_type,
                 regressor_kwargs,
                 client,
                 target_genes='all',
                 limit=None,
                 include_meta=False,
                 early_stop_window_length=EARLY_STOP_WINDOW_LENGTH,
                 repartition_multiplier=1,
                 seed=DEMON_SEED):
    """
    Main API function. Create a Dask computation graph.

    :param expression_matrix: numpy matrix. Rows are observations and columns are genes.
    :param gene_names: list of gene names. Each entry corresponds to the expression_matrix column with same index.
    :param tf_names: list of transcription factor names. Shoul

In [8]:
# client.close()

In [9]:
# %%time
# macosko_df = pd.read_csv(macosko_40k, sep='\t', dtype='float32')
# macosko_df = macosko_df.loc[:, (macosko_df != 0).any(axis=0)]  # remove all-0 columns

In [10]:
# macosko_df.to_parquet('macosko_40k_parquet')

In [8]:
%%time
macosko_df = pd.read_parquet('macosko_40k_parquet')

CPU times: user 7.65 s, sys: 9.6 s, total: 17.3 s
Wall time: 6.67 s


In [9]:
macosko_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 0 to 39999
Columns: 12934 entries, KITL to 6030443J06RIK
dtypes: float32(12934)
memory usage: 1.9 GB


In [ ]:
mm9_tfs = load_tf_names(tfs)

```
finished
CPU times: user 2h 44min 56s, sys: 2h 29min 37s, total: 5h 14min 33s
Wall time: 7h 14min 39s
```

In [ ]:
network_df = grnboost2(expression_data=macosko_df, 
                       tf_names=mm9_tfs, 
                       client_or_address=client,
                       verbose=True)

preparing dask client
parsing input
creating dask graph
88 partitions
computing dask graph


In [11]:
expression_matrix = macosko_df.as_matrix()
gene_names = list(macosko_df.columns)

In [12]:
graph = create_graph(expression_matrix,
                     gene_names,
                     tf_names=mm9_tfs,
                     client=client,
                     regressor_type='GBM',
                     regressor_kwargs=SGBM_KWARGS)

In [13]:
graph.npartitions

88

In [ ]:
network_df = client \
                .compute(graph, sync=True) \
                .sort_values(by='importance', ascending=False)

```
GRNBoost2 finished (float32 dataframe)
CPU times: user 1h 18min 15s, sys: 58min 48s, total: 2h 17min 4s
Wall time: 2h 37min 55s

GRNBoost2 finished (float64 dataframe)
CPU times: user 1h 17min 14s, sys: 57min 49s, total: 2h 15min 4s
Wall time: 2h 38min 53s
```

In [14]:
len(network_df)

2921508

In [17]:
macosko_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 12953 entries, KITL to 6030443J06RIK
dtypes: float64(12953)
memory usage: 3.9 GB


In [22]:
macosko_df_16 = macosko_df.astype('float16')

In [28]:
macosko_df_16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 12953 entries, KITL to 6030443J06RIK
dtypes: float16(12953)
memory usage: 988.2 MB


In [ ]:
%%time
network_df = grnboost2(expression_data=macosko_df_16,
                       tf_names=mm9_tfs,
                       client_or_address=client,
                       verbose=True)

preparing dask client
parsing input
creating dask graph
computing dask graph
